In [2]:
cd ../

c:\Users\eduar\Documents\usp\AMST\ml_timeseries_usp


In [46]:

from utils import read_all_csv_files
df = read_all_csv_files()
from utils import (
    read_all_csv_files,
    correcting_dtypes,
    stocks_removing_negatives,
    remove_materials_no_sales,
    plot_time_series_by_products,
    plot_products_materials,
    plot_produtos_in_cds,
    plot_time_series_cds_products_by_stores,
    plot_time_series_cds_products_by_store_size,
    plot_time_series_cds_products_by_store_weather,
    plot_time_series_cds_products_by_store_profile,
    plot_time_series_cds_products_store_region,
    violin_plot_material_store_sales,
    hierarchical_aggregation,
    remove_time_series_min_length
    )
import pandas as pd
import seaborn as sns
from statsforecast import StatsForecast
from statsforecast.models import MSTL,AutoETS,AutoARIMA,SeasonalExponentialSmoothing
from hierarchicalforecast.methods import BottomUp
from statsmodels.graphics.tsaplots import plot_acf
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import evaluate
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from utilsforecast.losses import rmse, mae, smape, mase, scaled_crps, mqloss,rmsse
from functools import partial
from prophet import Prophet
from utilsforecast.losses import rmse, mae, smape, mase, scaled_crps, mqloss
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import numpy as np
import aeon
#data preparation
df = read_all_csv_files()
df = correcting_dtypes(df)
df = stocks_removing_negatives(df)
df = remove_materials_no_sales(df)
df = remove_time_series_min_length(df, min_length=24)
df = df.drop_duplicates(['mes','sku','loja'])
df = df.sort_values(['sku','loja','mes']).reset_index(drop=True)[['sku','loja','mes','venda_unidades']]
import lightgbm as lgb

In [8]:
df['mes'].shift(6)

0               NaT
1               NaT
2               NaT
3               NaT
4               NaT
            ...    
846331   2024-11-01
846332   2024-11-01
846333   2024-11-01
846334   2024-11-01
846335   2024-11-01
Name: mes, Length: 846336, dtype: datetime64[ns]

In [ ]:
def make_time_features(df,date_col='mes'):
    df.sort_values(['material','sku'])
    #lags_cols
    for lag in [ 3, 6, 12, 18, 24]:
        df[f'lag({lag})'] = df['venda_unidades'].shift(lag)
    return df

In [45]:
df.sort_values(['sku','loja','mes']).reset_index(drop=True)[['sku','loja','mes','venda_unidades']]

,sku,loja,mes,venda_unidades
0,013b91d62d311c03b38be2149d7f096f,00703d74-ba0a-4dbf-81a1-605aff02efd7,2023-02-01,0.0
1,013b91d62d311c03b38be2149d7f096f,00703d74-ba0a-4dbf-81a1-605aff02efd7,2023-03-01,1.0
2,013b91d62d311c03b38be2149d7f096f,00703d74-ba0a-4dbf-81a1-605aff02efd7,2023-04-01,0.0
3,013b91d62d311c03b38be2149d7f096f,00703d74-ba0a-4dbf-81a1-605aff02efd7,2023-05-01,0.0
4,013b91d62d311c03b38be2149d7f096f,00703d74-ba0a-4dbf-81a1-605aff02efd7,2023-06-01,0.0
...,...,...,...,...
336248,f94a165a135c0b24c9a3f5add32c70f5,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,2025-03-01,0.0
336249,f94a165a135c0b24c9a3f5add32c70f5,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,2025-04-01,0.0
336250,f94a165a135c0b24c9a3f5add32c70f5,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,2025-05-01,0.0
336251,f94a165a135c0b24c9a3f5add32c70f5,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,2025-06-01,0.0


In [17]:
df = make_time_features(df)

In [35]:
pd.set_option('display.max_columns', None)

In [37]:
df.drop_duplicates(['mes','sku','loja'])

,mes,material,sku,produto,cor,tamanho,ordem_tamanho,construcao,padronagem,marca,decote,detalhe,loja,centro_distribuicao,idade_loja,uf_loja,cidade_loja,regiao_loja,bairro_loja,clima_loja,perfil_loja,porte_loja,estoque_unidades_inicio,estoque_unidades_fim,estoque_unidades,estoque_unidades_mediana,prop_dias_com_estoque,venda_unidades,venda_valor_total,preco_medio_unitario,preco_unitario_min,preco_unitario_max,variabilidade_preco_unitario,prop_dias_com_venda,total_observacoes_vendas
0,2023-08-01,3a5e87c5f227c2224fc7f24787d40b95,bb6e7aa5075d9460625318889e2fea62,calca,preto,pp,1,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0
1,2024-09-01,3a5e87c5f227c2224fc7f24787d40b95,6bdc4f2e3e6d9b2d321faa61e43c21b4,calca,preto,p,2,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0
2,2025-05-01,3a5e87c5f227c2224fc7f24787d40b95,141a82cc0285c5e19222ba3a621b92ac,calca,preto,gg,5,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,5.0,5.0,5.032258,5.0,1.0,1.0,159.99,159.99,159.99,159.99,NaN,0.032258,1
3,2023-02-01,3a5e87c5f227c2224fc7f24787d40b95,013b91d62d311c03b38be2149d7f096f,calca,preto,g,4,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0
4,2024-03-01,3a5e87c5f227c2224fc7f24787d40b95,6bdc4f2e3e6d9b2d321faa61e43c21b4,calca,preto,p,2,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,1.0,5.0,1.838710,2.0,1.0,1.0,139.90,139.90,139.90,139.90,NaN,0.032258,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846322,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,sem decote,sem manga,fa4f88d2-c689-4352-8579-b505860c133c,CD2,15,RJ,RIO DE JANEIRO,SUDESTE,BANGU,2,1,3,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0
846325,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,reto,sem manga,fceb9fd3-141c-467b-94df-34a9af611691,CD2,25,SP,SAO PAULO,SUDESTE,VILA BRANDINA,2,3,3,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0
846328,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,reto,sem manga,fd559418-7f7c-495c-a05e-242b13575f29,CD1,23,CE,CEARA,NORDESTE,SAO GERARDO,3,2,2,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0
846330,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,sem decote,sem manga,fdf39bea-2605-4363-848b-eabff9be6133,CD2,14,SP,SAO PAULO,SUDESTE,CERAMICA,1,3,3,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.000000,0


In [36]:
df[(df['mes'] == '2023-02-01') &	(df['sku'] == '013b91d62d311c03b38be2149d7f096f') &	(df['loja'] == '00703d74-ba0a-4dbf-81a1-605aff02efd7')]

,mes,material,sku,produto,cor,tamanho,ordem_tamanho,construcao,padronagem,marca,decote,detalhe,loja,centro_distribuicao,idade_loja,uf_loja,cidade_loja,regiao_loja,bairro_loja,clima_loja,perfil_loja,porte_loja,estoque_unidades_inicio,estoque_unidades_fim,estoque_unidades,estoque_unidades_mediana,prop_dias_com_estoque,venda_unidades,venda_valor_total,preco_medio_unitario,preco_unitario_min,preco_unitario_max,variabilidade_preco_unitario,prop_dias_com_venda,total_observacoes_vendas
3,2023-02-01,3a5e87c5f227c2224fc7f24787d40b95,013b91d62d311c03b38be2149d7f096f,calca,preto,g,4,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0
8,2023-02-01,3a5e87c5f227c2224fc7f24787d40b95,013b91d62d311c03b38be2149d7f096f,calca,preto,g,4,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,0.0,2.0,0.285714,0.0,0.142857,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0
14,2023-02-01,3a5e87c5f227c2224fc7f24787d40b95,013b91d62d311c03b38be2149d7f096f,calca,preto,g,4,tecido plano,liso,9,sem decote,longo,00703d74-ba0a-4dbf-81a1-605aff02efd7,CD1,14,BA,BAHIA,NORDESTE,GOES CALMON,3,2,1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0


In [24]:
df.sort_values(['mes','loja','sku'])[['mes','loja','sku','venda_unidades']]

,mes,loja,sku,venda_unidades
3,2023-02-01,00703d74-ba0a-4dbf-81a1-605aff02efd7,013b91d62d311c03b38be2149d7f096f,0.0
8,2023-02-01,00703d74-ba0a-4dbf-81a1-605aff02efd7,013b91d62d311c03b38be2149d7f096f,0.0
14,2023-02-01,00703d74-ba0a-4dbf-81a1-605aff02efd7,013b91d62d311c03b38be2149d7f096f,0.0
159784,2023-02-01,00703d74-ba0a-4dbf-81a1-605aff02efd7,0c57d0c28002f1493e44d35c6be0f12a,1.0
4636,2023-02-01,00703d74-ba0a-4dbf-81a1-605aff02efd7,0d69599ebe953a29c1b8c4fe37f1fb8a,0.0
...,...,...,...,...
297311,2025-07-01,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,f7be70137920694c947f62a68b080b2a,6.0
297319,2025-07-01,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,f7be70137920694c947f62a68b080b2a,0.0
297323,2025-07-01,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,f7be70137920694c947f62a68b080b2a,0.0
299818,2025-07-01,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,f94a165a135c0b24c9a3f5add32c70f5,0.0


In [ ]:
spec = [
        ['total'],
        ['total','centro_distribuicao'],
        #['total','centro_distribuicao','regiao_loja'],
        #['total','centro_distribuicao','regiao_loja','clima_loja'],
        #['total','centro_distribuicao','regiao_loja','clima_loja','loja'],
        #['total','centro_distribuicao','regiao_loja','clima_loja','loja','produto'],
        #['total','centro_distribuicao','regiao_loja','clima_loja','loja','produto','material'],
        #['total','centro_distribuicao','regiao_loja','clima_loja','loja','produto','material','sku']
]

Y_df, S_df, tags = hierarchical_aggregation(df, spec, columns = ['centro_distribuicao','regiao_loja','clima_loja','loja','produto','material','sku','mes','venda_unidades','estoque_unidades'], ds='mes', y='venda_unidades')